# 価格は適正

今日、商品の価格を推定するためのより複雑なソリューションを構築しています。

1. Day2.0ノートブック：RAG-DB作成
2. Day2.1ノートブック：2Dで視覚化
3. Day2.2ノートブック：3Dで視覚化
4. Day2.3ノートブック：GPT-4o miniでRAGパイプラインを構築してテスト
5. Day2.4ノートブック：(a) ランダムフォレストプライザー (b)アンサンブルプライザーを作成

うーん！それは一日で乗り越えるのにたくさんのことです！

## ご注意ください：

私たちはすでに、独自のファインチューニングされたLLMを使用して非常に強力な製品推定器を持っています。  

ほとんどの人はそれに非常に満足するでしょう！これらの追加の手順を追加する主な理由は、RAGとエージェント・ワークフローで専門知識を深めることです。


In [1]:
# import

# 基本的な
import os
import re
import math
import json
import random
import pickle

# HF
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset

# RAG
import chromadb
from sentence_transformers import SentenceTransformer

# 可視化
import numpy as np
from tqdm import tqdm
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# .envファイルから環境変数をロード

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore" # chromadbのDB名

In [3]:
# Hugging Faceにログイン

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [4]:
# items.py からの import - ありがとう、Trung N.！

from items import Item # 第6週の実装と全く同じ

## PKLファイルに戻ります

第6週にデータキュレーションを楽しんだように、おそらくそのプロセス全体を再び経験したくないでしょう！

次に作成したPKLファイルを再利用しましょう。`train.pkl` と `test.pkl` ファイルを今週の第8週フォルダにコピーするか、ここからダウンロードもできます。

https://drive.google.com/drive/folders/1f_IZGybvs9o0J5sb3xmtTEQB3BXllzrW?usp=drive_link

In [5]:
# このフォルダーにtrain.pklを配置すると、これを実行できる。

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [6]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7\n\nPrice is $227.00'

# 次に、Chroma DataStoreを作成します

第5週では、架空の企業Insurellmのオブジェクトチャンクを表す123個のドキュメントを含むChromaデータストアを作成しました。

次に、トレーニングデータセットから40万点の製品を含むChromaデータストアを作成します！いよいよ現実味を帯びてきました！

LangChainは使用しませんが、APIは非常にわかりやすく、以前のバージョンと一貫性があります。

特別なメモ：WindowsユーザーでChromaがクラッシュした場合は、次のコマンドでChromaライブラリを以前のバージョンにロールバックしてみてください。
`!pip install chromadb==0.5.0`

この問題を発見し、GitHubの問題[こちら](https://github.com/chroma-core/chroma/issues/2513)を指摘してくれた学生のKelly Z.に感謝します。
ただし、まずはChromaを元に戻さずに試してみてください。

In [7]:
client = chromadb.PersistentClient(path=DB)

In [8]:
# コレクションが存在する場合は削除
collection_name = "products"

# 古いバージョンのChromaの場合、後続行の代わりにこの行を使用
existing_collection_names = [collection.name for collection in client.list_collections()]
#existing_collection_names = client.list_collections()

if collection_name in existing_collection_names:
    print(f"Existing collection: {collection_name}")
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

Existing collection: products
Deleted existing collection: products


# sentenceTransfomerの紹介

All-Minilmは、文と段落を384次元密度のベクトル空間にマッピングするHugging Faceの非常に便利なモデルであり、セマンティック検索などのタスクに最適です。

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

ローカルでかなり迅速に実行できます。

前回、OpenAI埋め込みを使用してベクトル埋め込みを生成しました。OpenAI Embeddingsと比較した利点：

1. それは無料で速いです！
3. ローカルで実行できる - 個人的なRAG構築に役立つ可能性

In [9]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [10]:
# テキスト（のリスト）をベクトル（のリスト）化

vector = model.encode(["Well hi there"])[0]
vector

array([-9.46715996e-02,  4.27619927e-02,  5.51620461e-02, -5.10995043e-04,
        1.16203064e-02, -6.80130571e-02,  2.76406296e-02,  6.06974177e-02,
        2.88530905e-02, -1.74128544e-02, -4.94346432e-02,  2.30992641e-02,
       -1.28614204e-02, -4.31402475e-02,  2.17509363e-02,  4.26548757e-02,
        5.10500111e-02, -7.79727772e-02, -1.23247221e-01,  3.67456004e-02,
        4.54122620e-03,  9.47938487e-02, -5.53098917e-02,  1.70641635e-02,
       -2.92872861e-02, -4.47124429e-02,  2.06784345e-02,  6.39320239e-02,
        2.27427911e-02,  4.87789996e-02, -2.33501615e-03,  4.72859144e-02,
       -2.86258757e-02,  2.30624750e-02,  2.45130155e-02,  3.95681709e-02,
       -4.33176644e-02, -1.02316692e-01,  2.79873190e-03,  2.39303671e-02,
        1.61556490e-02, -8.99083540e-03,  2.07255818e-02,  6.40123338e-02,
        6.89179599e-02, -6.98361397e-02,  2.89760833e-03, -8.10990036e-02,
        1.71122737e-02,  2.50654924e-03, -1.06529079e-01, -4.87733372e-02,
       -1.67762246e-02, -

In [11]:
# クイックサイドバー - ビデオへの追加 - ベクターを比較する機能

import numpy as np

#  2つの数値ベクトル a, b のコサイン類似度（値域 -1〜1）を計算
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# テキスト text1, text2 を埋め込みモデルでベクトル化し、コサイン類似度を計算
def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2]) # ベクトル化
    similarity = cosine_similarity(vector1, vector2) # コサイン類似度
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%") # 結果

In [12]:
# そして、コンテキストにいくつかの単語を追加することで状況がどのように変化するかを見てみましょう。

how_similar("Java", "C++")        # PG言語的意味で近い。
how_similar("Java", "mug")        # PG言語？コーヒー？遠くなる。
how_similar("Cup of Java", "mug") # コーヒー的意味で近い。

Similarity between Java and C++ is 50.7%
Similarity between Java and mug is 25.8%
Similarity between Cup of Java and mug is 49.3%


In [13]:
# さて、本題に戻りましょう。ベクトル化できるものを作ってみましょう。

# プロンプトから「商品の説明文」だけを抽出
# ・冒頭の定型文を削除
# ・末端の価格行を削除
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

description(train[0])

'Delphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'

## RAGデータストアに入力

次のセルは、Chroma に 400,000 個のアイテムを入力。

処理に時間がかかりすぎる場合は、ドキュメント数を減らしても構いません。次のように変更できます。：  
`NUMBER_OF_DOCUMENTS = 20000`  
これで、RAG パイプラインを問題なく実行できます。

以下のセルの実行中に中断した場合は、再度実行する前に、Chroma DataStoreをクリアする必要がある場合があります（コレクションを削除する前のセルを再実行します）。そうしないと、同じ ID のドキュメントが既に存在するというエラーが発生します。

In [14]:
# 40万件の処理に（私のLaptopでは）8時間かかった。

batch_size = 1000 # バッチ処理数
NUMBER_OF_DOCUMENTS = len(train) # ドキュメント総数

# 400,000まで待たない場合はコメントを解除
# number_of_documents = 20000

# 進捗表示
pbar = tqdm(total=NUMBER_OF_DOCUMENTS, desc="Indexing", unit="doc")

for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, batch_size)):
    # プロンプトから「商品の説明文」だけを抽出
    documents = [description(item) for item in train[i: i+batch_size]]
    # 商品の説明文をベクトル化
    vectors = model.encode(documents).astype(float).tolist()
    # メタデータ（category、price）
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+batch_size]]
    # 一意なIDを連番で作成
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]

    # コレクションに一括追加
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

    pbar.update(len(documents))               # 追加した件数で進捗更新
    # pbar.set_postfix(batch=i // batch_size) # 任意: 現在のバッチ番号を表示

pbar.close()

Indexing: 100%|███████████████████████████████████████████████████████████████████████████| 400000/400000 [8:06:07<00:00, 13.71doc/s]
